<a href="https://colab.research.google.com/github/Real1CM/MLassignment/blob/main/%E8%AF%BE%E7%A8%8B%E6%8A%A5%E5%91%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 机器学习结课作业
#### 202408120021142 梁宸铭

#### 题目
##### 结合自己研究方向或兴趣，选择采用深度模型解决的应用问题。
##### 完成研究背景调研、方法说明、方法评价及后续展望。

#### 研究背景调研
人脸识别（Face Recognition）技术近年来成为计算机视觉与人工智能领域的研究热点。其研究背景涵盖了技术需求、算法发展、数据集和应用等多个方面。人脸识别是一种通过分析和比较面部特征来识别个体身份的生物特征识别技术。由于人脸在图像和视频中是公开的、非接触式的生物特征，相较于指纹、虹膜等识别方式，人脸识别应用广泛，且对用户的隐私影响较小。伴随着智能化需求的不断提升，人脸识别在安全、金融、医疗、社交等场景中已逐步成为身份验证的主流技术之一。因此，对人脸识别技术的高精度、低延迟、多场景适应性等要求不断提高，这也成为推动人脸识别技术发展的关键动力。早期的人脸识别依赖传统的图像处理与机器学习算法，如：
*   PCA（主成分分析）：Fisherfaces和Eigenfaces算法通过提取面部特征向量实现识别。
*   LBP（局部二值模式）：通过分析图像的纹理特征进行人脸检测与识别。

随着深度学习的兴起，基于卷积神经网络（CNN）的算法逐渐成为主流。2015年，Facebook推出的DeepFace模型使得人脸识别精度接近人类，推动了深度学习在该领域的快速应用。之后，FaceNet、VGGFace和ArcFace等模型进一步提升了人脸识别的精度和鲁棒性。现今，研究人员探索更加轻量级、高效且易于部署的模型（如MobileFaceNet、EfficientNet等）以适应移动设备的性能需求。



常用的标准数据集包括：

*   LFW（Labelled Faces in the Wild）：用于评估模型的跨姿态、光照变化下的鲁棒性。
*   MS-Celeb-1M：包含上百万张人脸图片，用于大规模模型训练。
*   MS-Celeb-1M：包含上百万张人脸图片，用于大规模模型训练。

GGFace2 和 CelebA：用于训练模型以提高对人脸表情、年龄变化的识别性能。
随着数据集规模和多样性的提升，人脸识别的评估标准也不断改进，从识别精度逐步发展到包含实时性、抗攻击性、跨年龄/性别/种族的公平性等多维度的指标。

当前，人脸识别的研究已从追求识别精度逐步转向效率、隐私和公平性方向。例如，联邦学习和隐私计算技术被应用于避免人脸数据直接传输，同时实现分布式模型训练；同时，可解释性和公平性的研究有助于识别和减少偏见。未来，人脸识别技术将进一步发展，向更智能、更安全、更具普适性的方向前进。

In [2]:
!unzip /content/files-for-face-verification-and-recognition.zip -d /content/files-for-face-verification-and-recognition/

Archive:  /content/files-for-face-verification-and-recognition.zip
  inflating: /content/files-for-face-verification-and-recognition/images/andrew.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/arnaud.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/benoit.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/bertrand.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_0.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_1.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_2.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_3.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_4.jpg  
  inflating: /content/files-for-face-verification-and-recognition/images/camera_5.jpg  
  inflating: /content/files-for-face-verification-and-recog

#### 方法说明
使用已经训练好的模型的权重去预测其他人脸数据，从而判断是不是对应的人物

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
import tf_keras as k3
model = k3.models.load_model('./files-for-face-verification-and-recognition/model')

In [4]:
def triplet_loss(y_true, y_pred, alpha = 0.2):

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))

    return loss

In [6]:
FRmodel = model

In [8]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [9]:
database = {}
database["danielle"] = img_to_encoding("./files-for-face-verification-and-recognition/images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("./files-for-face-verification-and-recognition/images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("./files-for-face-verification-and-recognition/images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("./files-for-face-verification-and-recognition/images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("./files-for-face-verification-and-recognition/images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("./files-for-face-verification-and-recognition/images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("./files-for-face-verification-and-recognition/images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("./files-for-face-verification-and-recognition/images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("./files-for-face-verification-and-recognition/images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("./files-for-face-verification-and-recognition/images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("./files-for-face-verification-and-recognition/images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("./files-for-face-verification-and-recognition/images/arnaud.jpg", FRmodel)

In [10]:
danielle = tf.keras.preprocessing.image.load_img("./files-for-face-verification-and-recognition/images/danielle.png", target_size=(160, 160))
kian = tf.keras.preprocessing.image.load_img("./files-for-face-verification-and-recognition/images/kian.jpg", target_size=(160, 160))

In [11]:
np.around(np.array(kian) / 255.0, decimals=12).shape

(160, 160, 3)

In [12]:
np.around(np.array(danielle) / 255.0, decimals=12).shape

(160, 160, 3)

因为人脸识别多数应用在门禁上，比如实验室门禁。故此处使用vertify函数来模拟开门关门，如果是正确的人就打印允许进入的信息来模拟门禁。

In [15]:
def verify(image_path, identity, database, model):

    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(tf.subtract(database[identity], encoding))
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
    return dist, door_open

In [16]:
distance, door_open_flag = verify("./files-for-face-verification-and-recognition/images/camera_0.jpg", "younes", database, FRmodel)
print("(", distance, ",", door_open_flag, ")")

It's younes, welcome in!
( 0.6019791 , True )


In [17]:
verify("./files-for-face-verification-and-recognition/images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(1.0130048, False)

In [18]:
def who_is_it(image_path, database, model):

    encoding =  img_to_encoding(image_path, model)

    min_dist = 100

    for (name, db_enc) in database.items():

        dist = np.linalg.norm(tf.subtract(db_enc, encoding))

        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity

In [19]:
who_is_it("./files-for-face-verification-and-recognition/images/camera_0.jpg", database, FRmodel)

test1 = who_is_it("./files-for-face-verification-and-recognition/images/camera_0.jpg", database, FRmodel)

test2 = who_is_it("./files-for-face-verification-and-recognition/images/younes.jpg", database, FRmodel)

it's younes, the distance is 0.6019791
it's younes, the distance is 0.6019791
it's younes, the distance is 0.0


#### 方法评价
利用训练好的模型来预测人脸，使用的训练集和测试集中图片是重合的，模型不能在没有预先训练的情况下做zero-shot或者few-shot的推理，这个人脸识别模型本身泛化能力欠缺，可以考虑利用多模态MutiModel来提高它的泛化能力。

#### 后续展望
人脸识别技术在快速发展中面临提升识别精度、实现多领域应用、增强隐私保护等挑战。未来的研究将着重在复杂环境下提升深度学习模型的识别能力，同时推动该技术在智慧城市、公共安全、个性化教育等领域的深入应用。随着隐私问题日益受到关注，加密和联邦学习等技术将帮助实现数据合规使用。此外，设备端的轻量化和低能耗模型也成为必要，以适应移动和物联网设备的需求。未来的人脸识别系统可能融合多模态信息，如声音、动作等，从而提高可靠性和适应性。同时，针对对抗攻击的防御技术也将不断优化，以保障系统的安全性。人脸识别技术的未来不仅在于识别精度和广泛应用，更在于实现智能、可信赖的发展路径。

附[完整示例](https://github.com/Real1CM/MLassignment)